<a href="https://colab.research.google.com/github/Esanchezponce/01_MIAR_OCT_24/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_(V2)_Erick_Sanchez_Ponce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Erick Hernan Sanchez Ponce  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---/tree/master/TrabajoPractico<br>
Google Colab: https://colab.research.google.com/drive/xxxxxxxxxxxxxxxx <br>
Problema:
>1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de una jornada de La Liga<br>
>3. Configuración de Tribunales

Descripción del problema:(copiar enunciado)

....







                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

In [ ]:
#Respuesta


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

In [ ]:
#Respuesta


#Diseño
- ¿Que técnica utilizo? ¿Por qué?

In [ ]:
#Respuesta

#Desarrollo

In [ ]:
import random
import pandas as pd
from collections import defaultdict

# Datos del problema (directamente desde la imagen)
data = {
    'Toma': range(1, 31),
    'Actor_1': [1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1],
    'Actor_2': [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
    'Actor_3': [1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
    'Actor_4': [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1],
    'Actor_5': [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1],
    'Actor_6': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    'Actor_7': [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    'Actor_8': [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
    'Actor_9': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
    'Actor_10': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
}
df_actores = pd.DataFrame(data)
df_actores = df_actores.set_index('Toma')

num_tomas = len(df_actores)
num_actores = len(df_actores.columns)
max_tomas_por_dia = 6

def calcular_costo_dia(tomas_dia, df):
    """Calcula el costo (número de actores únicos) para un día de grabación."""
    actores_requeridos = [0] * num_actores
    for toma_index in tomas_dia:
        for i, col in enumerate(df.columns):
            if df.loc[toma_index, col] == 1:
                actores_requeridos[i] = 1
    return sum(actores_requeridos)

def evaluar_solucion(solucion, df):
    """Evalúa una solución (diccionario de día: [lista de tomas]) y devuelve su costo."""
    costo_total = 0
    for dia, tomas_en_dia in solucion.items():
        if len(tomas_en_dia) > max_tomas_por_dia:
            return float('inf')  # Penalización por exceder el límite de tomas
        costo_total += calcular_costo_dia(tomas_en_dia, df)
    return costo_total

def generar_vecinos(solucion_actual, tomas_pendientes_inicial):
    """Genera soluciones vecinas moviendo una toma a otro día."""
    vecinos = []
    for dia_origen, tomas_origen in solucion_actual.items():
        for i, toma_a_mover in enumerate(tomas_origen):
            # Intentar mover la toma a un día existente
            for dia_destino, tomas_destino in solucion_actual.items():
                if dia_origen != dia_destino and len(tomas_destino) < max_tomas_por_dia:
                    nueva_solucion = solucion_actual.copy()
                    nueva_solucion[dia_origen] = tomas_origen[:i] + tomas_origen[i+1:]
                    if not nueva_solucion[dia_origen]:
                        del nueva_solucion[dia_origen]
                    nueva_solucion[dia_destino] = tomas_destino + [toma_a_mover]
                    vecinos.append(nueva_solucion)

            # Intentar mover la toma a un nuevo día (si no todas las tomas están asignadas)
            tomas_asignadas = set()
            for tomas in solucion_actual.values():
                tomas_asignadas.update(tomas)
            if len(tomas_asignadas) < len(tomas_pendientes_inicial):
                nuevo_dia = max(solucion_actual.keys()) + 1 if solucion_actual else 1
                nueva_solucion = solucion_actual.copy()
                nueva_solucion[dia_origen] = tomas_origen[:i] + tomas_origen[i+1:]
                if not nueva_solucion[dia_origen]:
                    del nueva_solucion[dia_origen]
                nueva_solucion[nuevo_dia] = [toma_a_mover]
                vecinos.append(nueva_solucion)
    return vecinos

def generar_solucion_inicial(num_tomas, max_tomas_por_dia):
    """Genera una solución inicial aleatoria."""
    tomas = list(range(1, num_tomas + 1))
    random.shuffle(tomas)
    solucion = defaultdict(list)
    dia = 1
    for toma in tomas:
        solucion[dia].append(toma)
        if len(solucion[dia]) >= max_tomas_por_dia and toma != tomas[-1]:
            dia += 1
    return dict(solucion)

def busqueda_tabu(df, num_iteraciones, tamano_tabu):
    """Implementación del algoritmo de búsqueda tabú."""
    solucion_actual = generar_solucion_inicial(num_tomas, max_tomas_por_dia)
    mejor_solucion = solucion_actual.copy()
    mejor_costo = evaluar_solucion(mejor_solucion, df)
    lista_tabu = [tuple(sorted(item) for item in mejor_solucion.items())] # Almacenar soluciones como tuplas ordenadas

    tomas_pendientes_inicial = set(range(1, num_tomas + 1))

    for iteracion in range(num_iteraciones):
        vecinos = generar_vecinos(solucion_actual, tomas_pendientes_inicial)
        mejor_vecino = None
        mejor_costo_vecino = float('inf')

        for vecino in vecinos:
            costo_vecino = evaluar_solucion(vecino, df)
            vecino_tupla = tuple(sorted(item) for item in vecino.items())

            if costo_vecino < mejor_costo_vecino and vecino_tupla not in lista_tabu:
                mejor_costo_vecino = costo_vecino
                mejor_vecino = vecino
            elif costo_vecino < mejor_costo and vecino_tupla in lista_tabu:
                # Criterio de aspiración: permitir un movimiento tabú si mejora la mejor solución encontrada
                mejor_costo_vecino = costo_vecino
                mejor_vecino = vecino

        if mejor_vecino:
            solucion_actual = mejor_vecino
            costo_actual = mejor_costo_vecino
            vecino_tupla = tuple(sorted(item) for item in solucion_actual.items())
            lista_tabu.append(vecino_tupla)
            if len(lista_tabu) > tamano_tabu:
                lista_tabu.pop(0)

            if costo_actual < mejor_costo:
                mejor_costo = costo_actual
                mejor_solucion = solucion_actual.copy()

        print(f"Iteración {iteracion + 1}: Mejor costo actual = {mejor_costo}")

    return mejor_solucion, mejor_costo

# --- Ejecución de la búsqueda tabú ---
num_iteraciones = 500
tamano_tabu = 50

mejor_plan, mejor_costo = busqueda_tabu(df_actores, num_iteraciones, tamano_tabu)

print("\nMejor solución encontrada por Búsqueda Tabú:")
for dia, tomas in sorted(mejor_plan.items()):
    actores_dia = calcular_costo_dia(tomas, df_actores)
    print(f"Día {dia}: Tomas {tomas}, Actores requeridos: {actores_dia}")

print(f"\nCosto Total de Actores: {mejor_costo}")